In [1]:
import torch
import torchvision
import torch.nn as nn

In [2]:
data_path = "/Users/leo/Desktop/subjects_0/"

clone_path = "/Users/leo/Desktop/Thesis/data/subjects_copy/"

In [3]:
import shutil

import os
import random
import shutil

def copy_folder(src_path, dst_path, num_folders_keep):
    try:
        shutil.rmtree(dst_path)
    except FileNotFoundError:
        pass
    except Exception as e:
        print(f"Error deleting {dst_path}: {e}")
        return
    
    try:
        os.makedirs(dst_path, exist_ok=True)
    except Exception as e:
        print(f"Error creating {dst_path}: {e}")
        return
    
    try:
        all_folders = os.listdir(src_path)
        num_folders = len(all_folders)
        keep_folders = set(random.sample(all_folders, min(num_folders_keep, num_folders)))
        
        for folder in keep_folders:
            src_folder = os.path.join(src_path, folder)
            dst_folder = os.path.join(dst_path, folder)
            shutil.copytree(src_folder, dst_folder)
            print(f"Copied {src_folder} to {dst_folder}")
            
        print(f"Successfully copied {num_folders_keep} out of {num_folders} folders from {src_path} to {dst_path}")
        
    except Exception as e:
        print(f"Error copying folders from {src_path} to {dst_path}: {e}")


In [4]:
import os
import shutil
import random

import os
import random
import shutil

def split_dataset(src_folder, dst_folder, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    # Create destination directories
    train_dir = os.path.join(dst_folder, 'train')
    val_dir = os.path.join(dst_folder, 'val')
    test_dir = os.path.join(dst_folder, 'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Iterate over each subdirectory in the source folder
    for subdir_name in os.listdir(src_folder):
        subdir = os.path.join(src_folder, subdir_name)
        if not os.path.isdir(subdir):
            continue
        
        # Create subdirectories in the destination folders
        train_subdir = os.path.join(train_dir, subdir_name)
        val_subdir = os.path.join(val_dir, subdir_name)
        test_subdir = os.path.join(test_dir, subdir_name)
        os.makedirs(train_subdir, exist_ok=True)
        os.makedirs(val_subdir, exist_ok=True)
        os.makedirs(test_subdir, exist_ok=True)

        # Get a list of all files in the subdirectory
        files = os.listdir(subdir)
        num_files = len(files)

        # Shuffle the list of files randomly
        random.shuffle(files)

        # Calculate the number of files for each split
        num_train_files = int(num_files * train_ratio)
        num_val_files = int(num_files * val_ratio)
        num_test_files = int(num_files * test_ratio)

        # Split the files into train, validation, and test sets
        train_files = files[:num_train_files]
        val_files = files[num_train_files:num_train_files + num_val_files]
        test_files = files[num_train_files + num_val_files:num_train_files + num_val_files + num_test_files]

        # Copy the files to the appropriate directories
        for file in train_files:
            src_path = os.path.join(subdir, file)
            dst_path = os.path.join(train_subdir, file)
            shutil.copy(src_path, dst_path)

        for file in val_files:
            src_path = os.path.join(subdir, file)
            dst_path = os.path.join(val_subdir, file)
            shutil.copy(src_path, dst_path)

        for file in test_files:
            src_path = os.path.join(subdir, file)
            dst_path = os.path.join(test_subdir, file)
            shutil.copy(src_path, dst_path)

    print('Done splitting dataset.')

In [51]:
import sys

sys.path.append("/Users/leo/Desktop/Thesis/utils/")
sys.path.append("/Users/leo/Desktop/Thesis/models/")

import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
from torchvision.models import mobilenet_v3_large, mobilenet_v3_small
from torchvision.models import MobileNet_V3_Large_Weights as weights_large, MobileNet_V3_Small_Weights as weights_small
import torchvision.transforms as transforms
from torchinfo import summary
from helper_functions import delete_ds_store, plot_metrics, test, train, set_device, mean_std_finder, load_model
from CBAM import _CBAM
from MBNV3_2 import MBNV3Creator
from config import comet_token, project_name, workspace
from comet_ml import Experiment

#setting the seed for reproducibility
torch.manual_seed(42)

In [52]:
prms = {
    "backbone": mobilenet_v3_small,
    "weights": weights_small.IMAGENET1K_V1,
    "module": _CBAM,
    "module_alias": "CBAM",
    "data_path": "../data/faces_50/",
    "batch_size": 16,
    "img_size": 224,
    "num_classes": 50,
    "epochs": 5,
    "learning_rate": 0.001,
    "device": set_device(),
    "model": None,
    "experiment": None,
    "load_model": False,
}
prms["model_save_path"] = f"./output/{prms['module_alias']}_{prms['data_path'].split('/')[-2]}.pth"

model = prms["model"] = MBNV3Creator(prms["backbone"], prms["num_classes"], prms["weights"], prms["device"], prms["module"])
delete_ds_store(prms["data_path"])

In [53]:
#mean, std = mean_std_finder(prms["data_path"])
mean, std = torch.tensor([0.3860, 0.3119, 0.2690]), torch.tensor([0.2279, 0.2167, 0.2135])

train_transform = transforms.Compose([
    transforms.Resize((prms["img_size"], prms["img_size"])),
    transforms.RandAugment(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transform = transforms.Compose([
    transforms.Resize((prms["img_size"], prms["img_size"])),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [54]:
#Accessing the data
train_data = torchvision.datasets.ImageFolder(prms["data_path"] + "train/", transform=train_transform)
val_data = torchvision.datasets.ImageFolder(prms["data_path"] + "val/", transform=test_transform)
test_data = torchvision.datasets.ImageFolder(prms["data_path"] + "test/", transform=test_transform)

#Creating the data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=0)

In [69]:
from torchvision.models import resnet18

class res18(nn.Module):
    def __init__(self):
        super(res18, self).__init__()
        self.resnet = resnet18(pretrained=True)
        self.model_children = list(self.resnet.children())
        self.feature_conv = nn.Sequential(*self.model_children[:-2])
        self.classifier = nn.Sequential(*self.model_children[-2:])

        self.gradients = None

        del self.resnet

    def activations_hook(self, grad):
        self.gradients = grad

    def forward(self, x):
        x = self.feature_conv(x)
        print(x.shape)
        x.register_hook(self.activations_hook)
        print(x.shape)
        x = self.classifier(x)
        return x
    
    def get_activations_gradient(self):
        return self.gradients
    
    def get_activations(self, x):
        return self.feature_conv(x)

In [70]:
model = res18()

model.eval()

img, _ = next(iter(test_loader))

img = img[0].unsqueeze(0)


pred = model(img)


torch.Size([1, 512, 7, 7])
torch.Size([1, 512, 7, 7])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x1 and 512x1000)